In [84]:
import math;
import numpy as np;
import cv2

In [85]:
def DarkChannel(im,sz):
    b,g,r = cv2.split(im)
    dc = cv2.min(cv2.min(r,g),b);
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(sz,sz))
    dark = cv2.erode(dc,kernel)
    return dark

In [86]:
def AtmLight(im,dark):
    [h,w] = im.shape[:2]
    imsz = h*w
    
    
    imvec = im.reshape(imsz,3);
    
    #Original Old way for DCP 1
    numpx = int(max(math.floor(imsz/1000),1))
    darkvec = dark.reshape(imsz,1);
    indices = darkvec.argsort();
    indices = indices[imsz-numpx::]

    atmsum = np.zeros([1,3])
    for ind in range(1,numpx):
       atmsum = atmsum + imvec[indices[ind]]

    A1_old = atmsum / numpx;
    
    #New Way with 0.1 percent from DCP 2
    darkvec = dark.ravel()
    count = math.ceil(h * w * 0.1 / 100)
    indices = np.argpartition(darkvec, -count)[:-count]
    A2_new = np.amax(np.take(imvec, indices, axis=0), axis=0)
    
    A2_new = np.reshape(A2_new,A1_old.shape)
    
    print(A2_new) # more
    print(A1_old) # less
    print((A2_new+A1_old)/2)
    print("Next")
    
    return ((A2_new+A1_old)/2)

In [87]:
def TransmissionEstimate(im,A,sz):
    omega = 0.95;
    im3 = np.empty(im.shape,im.dtype);

    for ind in range(0,3):
        im3[:,:,ind] = im[:,:,ind]/A[0,ind]

    transmission = 1 - omega*DarkChannel(im3,sz);
    return transmission

In [88]:
def Guidedfilter(im,p,r,eps):
    mean_I = cv2.boxFilter(im,cv2.CV_64F,(r,r));
    mean_p = cv2.boxFilter(p, cv2.CV_64F,(r,r));
    mean_Ip = cv2.boxFilter(im*p,cv2.CV_64F,(r,r));
    cov_Ip = mean_Ip - mean_I*mean_p;

    mean_II = cv2.boxFilter(im*im,cv2.CV_64F,(r,r));
    var_I   = mean_II - mean_I*mean_I;

    a = cov_Ip/(var_I + eps);
    b = mean_p - a*mean_I;

    mean_a = cv2.boxFilter(a,cv2.CV_64F,(r,r));
    mean_b = cv2.boxFilter(b,cv2.CV_64F,(r,r));

    q = mean_a*im + mean_b;
    return q;

In [89]:
def TransmissionRefine(im,et):
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY);
    gray = np.float64(gray)/255;
    r = 60;
    eps = 0.0001;
    t = Guidedfilter(gray,et,r,eps);

    return t;

In [90]:
def Recover(im,t,A,tx = 0.1):
    res = np.empty(im.shape,im.dtype);
    t = cv2.max(t,tx);

    for ind in range(0,3):
        res[:,:,ind] = (im[:,:,ind]-A[0,ind])/t + A[0,ind]

    return res

In [91]:
if __name__ == '__main__':
    import sys
    
    for i in range(46,51):
        fn = './ValidationHazy/' + str(i)+ '.png'

        def nothing(*argv):
            pass

        src = cv2.imread(fn);

        I = src.astype('float64')/255;
 
        dark = DarkChannel(I,15);
        A = AtmLight(I,dark);
        te = TransmissionEstimate(I,A,15);
        t = TransmissionRefine(src,te);
        J = Recover(I,t,A,0.1);
      
        cv2.imwrite("./Submit/"+str(i)+".png",J*255);
    
 

[[0.90980392 0.86666667 0.84313725]]
[[0.85837623 0.79566381 0.76822712]]
[[0.88409007 0.83116524 0.80568219]]
Next
[[0.8745098  0.81568627 0.80392157]]
[[0.83093954 0.76038807 0.73295139]]
[[0.85272467 0.78803717 0.76843648]]
Next
[[0.63921569 0.62745098 0.65882353]]
[[0.61928513 0.58792892 0.57225082]]
[[0.62925041 0.60768995 0.61553717]]
Next
[[0.64313725 0.65882353 0.67058824]]
[[0.56441176 0.56833129 0.56049224]]
[[0.60377451 0.61357741 0.61554024]]
Next
[[0.92941176 0.92941176 0.92941176]]
[[0.14894199 0.14110294 0.13718342]]
[[0.53917688 0.53525735 0.53329759]]
Next
